<a href="https://colab.research.google.com/github/yardenahirsch/cs194W_test/blob/master/Parking_Spot_Detection_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Parking Spot Detection Model
#### We use a TensorFlow Hub Object Detection Model to identify and count the number of cars in a video frame. In this approach, we need to count the number of parking spots in a camera view when we set up the camera. Then we would repeatedly count the number of cars in a video frame and subtract this from the total number of parking spots to get the number of available parking spots. We run object detection on a video frame every 30 seconds, and it takes approximately 1.5 seconds to run object detection on each video frame. 

#### Notes: 
* You need to upload the video file "parking_lot.mov" to the files on the left side bar. The files get deleted each time you run this notebook. 
* It takes a few minutes to run the import statements but we would only need to do this once if we are running this continuously. 
* You can ignore the errors and warnings from running import statements.

##### Copyright 2018 The TensorFlow Hub Authors.

Licensed under the Apache License, Version 2.0 (the "License");

In [0]:
# Copyright 2018 The TensorFlow Hub Authors. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# ==============================================================================

## Imports


In [0]:
# Runs with stable version tensorflow 2.1.0.

!pip install tensorflow==2.1.0

# For running inference on the TF-Hub module.
import tensorflow as tf

import tensorflow_hub as hub

# For converting video to video frames 
import cv2

# For downloading the image.
import matplotlib.pyplot as plt
import tempfile
from six.moves.urllib.request import urlopen
from six import BytesIO

# For drawing onto the image.
import numpy as np
from PIL import Image
from PIL import ImageColor
from PIL import ImageDraw
from PIL import ImageFont
from PIL import ImageOps

# For measuring the inference time.
import time

## Constants

In [0]:
FPS = 60 # Video is FPS frames per second (depends on video)
CAPTURE_TIME_FREQUENCY = 30 # Capture a frame every CAPTURE_TIME_FREQUENCY seconds 
CAPTURE_FRAME_FREQUENCY = FPS*CAPTURE_TIME_FREQUENCY # Capture a frame at multiples of this value 
CAR_LABEL = b'Car' # Name used by object detection model for cars
MIN_SCORE_CAR = .50 # Minimum score required to count an object as a car 

## Helper functions for visualization.

Visualization code adapted from [TF object detection API](https://github.com/tensorflow/models/blob/master/research/object_detection/utils/visualization_utils.py) for the simplest required functionality.

In [0]:
def draw_bounding_box_on_image(image,
                               ymin,
                               xmin,
                               ymax,
                               xmax,
                               color,
                               font,
                               thickness=2,
                               display_str_list=()):
  """Adds a bounding box to an image."""
  draw = ImageDraw.Draw(image)
  im_width, im_height = image.size
  (left, right, top, bottom) = (xmin * im_width, xmax * im_width,
                                ymin * im_height, ymax * im_height)
  draw.line([(left, top), (left, bottom), (right, bottom), (right, top),
             (left, top)],
            width=thickness,
            fill=color)

  # If the total height of the display strings added to the top of the bounding
  # box exceeds the top of the image, stack the strings below the bounding box
  # instead of above.
  display_str_heights = [font.getsize(ds)[1] for ds in display_str_list]
  # Each display_str has a top and bottom margin of 0.05x.
  total_display_str_height = (1 + 2 * 0.05) * sum(display_str_heights)

  if top > total_display_str_height:
    text_bottom = top
  else:
    text_bottom = bottom + total_display_str_height
  # Reverse list and print from bottom to top.
  for display_str in display_str_list[::-1]:
    text_width, text_height = font.getsize(display_str)
    margin = np.ceil(0.05 * text_height)
    draw.rectangle([(left, text_bottom - text_height - 2 * margin),
                    (left + text_width, text_bottom)],
                   fill=color)
    draw.text((left + margin, text_bottom - text_height - margin),
              display_str,
              fill="black",
              font=font)
    text_bottom -= text_height - 2 * margin


def draw_boxes(image, boxes, class_names, scores):
  BOX_LABEL_COLOR = "white"
  """Overlay labeled boxes on an image with formatted scores and label names."""
  try:
    font = ImageFont.truetype("/usr/share/fonts/truetype/liberation/LiberationSansNarrow-Regular.ttf",
                              25)
  except IOError:
    print("Font not found, using default font.")
    font = ImageFont.load_default()

  for i in range(boxes.shape[0]):
    ymin, xmin, ymax, xmax = tuple(boxes[i])
    display_str = "{}: {}%".format(class_names[i].decode("ascii"),
                                    int(100 * scores[i]))
    image_pil = Image.fromarray(np.uint8(image)).convert("RGB")
    draw_bounding_box_on_image(
        image_pil,
        ymin,
        xmin,
        ymax,
        xmax,
        BOX_LABEL_COLOR,
        font,
        display_str_list=[display_str])
    np.copyto(image, np.array(image_pil))
  return image

## Model to detect cars in an image

Pick an object detection module and apply on the downloaded image. Modules:
* **FasterRCNN+InceptionResNet V2**: high accuracy,
* **ssd+mobilenet V2**: small and fast.

In [0]:
module_handle = "https://tfhub.dev/google/faster_rcnn/openimages_v4/inception_resnet_v2/1" #@param ["https://tfhub.dev/google/openimages_v4/ssd/mobilenet_v2/1", "https://tfhub.dev/google/faster_rcnn/openimages_v4/inception_resnet_v2/1"]

detector = hub.load(module_handle).signatures['default']

In [0]:
def load_img(path):
  img = tf.io.read_file(path)
  img = tf.image.decode_jpeg(img, channels=3)
  return img

In [0]:
def save_detected_image(image, original_filename):
  labeled_filename = "labeled_" + original_filename 
  plt.figure(figsize=(15,20))
  plt.imshow(image)
  plt.savefig(labeled_filename)

In [0]:
def run_detector(detector, path):
  img = load_img(path)

  converted_img  = tf.image.convert_image_dtype(img, tf.float32)[tf.newaxis, ...]
  start_time = time.time()
  result = detector(converted_img)
  end_time = time.time()

  result = {key:value.numpy() for key,value in result.items()} # This is a dictionary of detection class labels, detection class entities, and detection scores

  class_entities = result['detection_class_entities'] 
  class_scores = result['detection_scores']
  # Get a numpy array of bools, where True signifies that the object detected is a car and the object detection score is greater than the minimum score
  filter_cars = (class_entities == CAR_LABEL) & (class_scores > MIN_SCORE_CAR)
  num_cars = np.sum(filter_cars)

  elapsed_time = end_time-start_time
  print("%s: Detected %d cars in %f seconds" % (path, num_cars, elapsed_time))

  # Only draw boxes on cars that have a score greater than the minimum score
  image_with_boxes = draw_boxes(
      img.numpy(), result["detection_boxes"][filter_cars],
      result["detection_class_entities"][filter_cars], result["detection_scores"][filter_cars])

  save_detected_image(image_with_boxes, path)

## Run detection on a video 

In [0]:
def run_detection_on_video(video_filename):
  vidcap = cv2.VideoCapture(video_filename)
  success, image = vidcap.read()
  count = 0
  while success:
    # Only write and process the video frame if it is a multiple of the capture frame frequency 
    if count % CAPTURE_FRAME_FREQUENCY == 0: 
      frame_name = "frame%d.jpg" % count
      cv2.imwrite(frame_name, image) # save frame as JPEG file
      run_detector(detector, frame_name)
    success, image = vidcap.read()
    count += 1

In [0]:
run_detection_on_video("parking_lot.mov")